In [ ]:
import pandas as pd
import json
import re
import json
from bs4 import BeautifulSoup
import bs4
import requests
import os
from tqdm import tqdm
import pickle
from law_query import LawQuery,Tree
from slugify import slugify
import gzip

In [ ]:
# # get luat hien hanh
# req = requests.get('https://thuvienphapluat.vn/chinh-sach-phap-luat-moi/vn/thoi-su-phap-luat/tu-van-phap-luat/42248/danh-muc-luat-bo-luat-hien-hanh-tai-viet-nam')
# soup = BeautifulSoup(req.text, 'html.parser')
# luat_hien_hanh = soup.select('.newcontent a')
# urls = []
# for i in luat_hien_hanh:
#     urls.append(i['href'])
# urls = urls[1:]
# # save to urls.txt
# with open('urls.txt', 'w') as f:
#     f.write('\n'.join(urls))

In [ ]:
urls = []
with open('urls.txt', 'r') as f:
    urls = f.read().split('\n')

In [ ]:
def get_text(url):
    # get metadata
    metadata_url='https://thuvienphapluat.vn/AjaxLoadData/LoadLuocDo.aspx?LawID={}&IstraiNghiem=True'
    id = url.replace('.aspx','').split('-')[-1]
    req=requests.get(metadata_url.format(id),headers={'referer': url})
    if req.status_code != 200:
        raise Exception('Error when get metadata')
    soup = BeautifulSoup(req.text, 'html.parser')
    metadata = soup.select_one('#viewingDocument')
    metadata = {
        'ten_van_ban': metadata.select_one('#viewingDocument > div:nth-child(1)').getText(strip=True), 
        'so_hieu_van_ban': metadata.select_one('#viewingDocument > div:nth-child(2) > div.ds.fl').getText(strip=True),
        'loai_van_ban': metadata.select_one('#viewingDocument > div:nth-child(3) > div.ds.fl').getText(strip=True),
        'linhvuc': metadata.select_one('#viewingDocument > div:nth-child(4) > div.ds.fl').getText(strip=True),
        'noi_ban_hanh': metadata.select_one('#viewingDocument > div:nth-child(5) > div.ds.fl').getText(strip=True),
        'nguoi_ky': metadata.select_one('#viewingDocument > div:nth-child(6) > div.ds.fl').getText(strip=True),
        'ngay_ban_hanh': metadata.select_one('#viewingDocument > div:nth-child(7) > div.ds.fl').getText(strip=True),
        'ngay_hieu_luc': metadata.select_one('#viewingDocument > div:nth-child(8) > div.ds.fl').getText(strip=True),
        'ngay_cong_bao': metadata.select_one('#viewingDocument > div:nth-child(9) > div.ds.fl').getText(strip=True),
        'so_cong_bao': metadata.select_one('#viewingDocument > div:nth-child(10) > div.ds.fl').getText(strip=True),
        'tinh_trang': metadata.select_one('#viewingDocument > div:nth-child(11) > div.ds.fl').getText(strip=True),
    }
    req=requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    html = soup.find('div', {'class':'content1'})
    for element in html(text=lambda text: isinstance(text, bs4.Comment)):
        element.extract()
    for a in html.find_all('a', href=lambda x: x and x[0] == '#'):
        a.extract()
    for s in html.find_all(['script','style']):
        s.extract()
    ps = html.find_all(['p','h1','h2','h3','h4','h5','h6'])
    content = ''
    for p in ps:
        text = p.text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        content += text + '\n'
    # delete - more than 3
    content = re.sub(r'-{3,}', '', content)
    content = re.sub(r'\*{3,}', '', content)
    # remove line with empty content
    content = re.sub(r'^\s+$', '', content, flags=re.MULTILINE)
    # name = soup.find('h1').text.strip()
    matches = re.finditer(r'^(chương [\d\w]+.*|phần thứ [\d\w]+.*)$', content, re.MULTILINE | re.IGNORECASE)
    for i,match in enumerate(matches):
        end = match.end()
        while end < len(content) and content[end] != '\n':
            end += 1
        if end < len(content) and content[end] == '\n':
            content = content[:end] + ':' + content[end+1:]
    return metadata,content,html.prettify()

In [ ]:
# items = []
# for url in urls:
#     try:
#         metadata,text,html = get_text(url)
#         print(metadata['ten_van_ban'])
#         name_slug = slugify(metadata['ten_van_ban'])[:50]
#         items.append({'metadata':json.dumps(metadata),'path':name_slug})
#         folder_path = os.path.join('documents', name_slug)
#         os.makedirs(folder_path, exist_ok=True)
#         with open(os.path.join(folder_path, 'content.txt'), 'w', encoding='utf-8') as f:
#             f.write(text)
#         with open(os.path.join(folder_path, 'content.html'), 'w', encoding='utf-8') as f:
#             f.write(html)
#     except Exception as e:
#         print(e)
#         print(url)
#         continue
# df = pd.DataFrame(items)
# df.to_csv('documents/data.csv', index=False)

In [ ]:
def make_content_for_tree(text):
    re_footer = r'^(Bộ luật này đã được Quốc hội .*|Luật này được Quốc hội.*|Bộ luật này được Quốc hội .*|Luật này đã được Quốc hội .*)'
    matches = re.split(re_footer, text, flags=re.MULTILINE)
    text = matches[0]
    return text

In [ ]:
CREATE_CONTENT_TREE=False
df = pd.read_csv('documents/data.csv')
for row in df.itertuples():
    print(row.path)
    folder_path = os.path.join('documents', row.path)
    text = open(os.path.join(folder_path, 'content.txt'), encoding='utf-8').read()
    raw_text = text
    text = make_content_for_tree(text)
    if CREATE_CONTENT_TREE:
        with open(os.path.join(folder_path, 'content_tree.txt'), 'w', encoding='utf-8') as f:
            f.write(text)
    text = open(os.path.join(folder_path, 'content_tree.txt'), encoding='utf-8').read()
    tree = Tree(metadata= json.loads(row.metadata),content=text,raw_text=raw_text)
    with open(os.path.join(folder_path, 'debug_tree.txt'), 'w', encoding='utf-8') as f:
        f.write(str(tree))
    with open(os.path.join(folder_path, 'tree.pkl'), 'wb') as f:
        pickle.dump(tree, f)
    with open(os.path.join(folder_path, 'tree.json'), 'w', encoding='utf-8') as f:
        f.write(json.dumps(tree.export(), indent=4, ensure_ascii=False))

    with open('./law_query/documents/{}.json.gz'.format(row.path), 'wb') as f:
        f.write(gzip.compress(json.dumps(tree.export(), ensure_ascii=False).encode('utf-8')))

In [1]:
from law_query_public import Engine,list_documents

In [2]:
documents = list_documents()
query_engine = Engine(documents[0])
query_engine.query(node_type='phần')

[Phần thứ nhất:QUY ĐỊNH CHUNG,
 Phần thứ hai:QUYỀN SỞ HỮU VÀ QUYỀN KHÁC ĐỐI VỚI TÀI SẢN,
 Phần thứ ba:NGHĨA VỤ VÀ HỢP ĐỒNG,
 Phần thứ tư:THỪA KẾ,
 Phần thứ năm:PHÁP LUẬT ÁP DỤNG ĐỐI VỚI QUAN HỆ DÂN SỰ CÓ YẾU TỐ NƯỚC NGOÀI,
 Phần thứ sáu:ĐIỀU KHOẢN THI HÀNH]

In [3]:
query_engine.query_by_paths([
    {
        'node_type': 'phần',
        'node_id': 'nhất',
    },
    {
        'node_type': 'chương',
        'node_id': 'V',
    }
])

[Chương V:NHÀ NƯỚC CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM, CƠ QUAN NHÀ NƯỚC Ở TRUNG ƯƠNG, Ở ĐỊA PHƯƠNG TRONG QUAN HỆ DÂN SỰ]

In [4]:
query_engine

ten_van_ban: Bộ luật dân sự 2015
so_hieu_van_ban: 91/2015/QH13
loai_van_ban: Luật
linhvuc: Quyền dân sự
noi_ban_hanh: Quốc hội
nguoi_ky: Nguyễn Sinh Hùng
ngay_ban_hanh: 24/11/2015
ngay_hieu_luc: 01/01/2017
ngay_cong_bao: 28/12/2015
so_cong_bao: Từ số 1243 đến số 1244
tinh_trang: Còn hiệu lực